# Определение параметров интернет-пользователей, влияющих на выполнение ими целевого действия при просмотре сайта 

                        Автор материала: Максим Сапрыкин
                        
План статьи
1. Введение и постановка задачи
2. Описание набора данных и признаков                         

# 1. Введение и постановка задачи

В течение последних двух десятилетий в мире и в частности в нашей стране гармонично развивались совершенно новые профессии, связанные с бурной информатизацией общества и распространением сети Интернет. В частности, появились специалисты, занимающиеся электронной комерцией, продажей и продвижением продуктов (товаров или услуг) в интернете - интернет-маркетологи (или иногда digital-маркетологи). 

С развитием различных каналов привлечение интернет-трафика, таких как системы контекстной рекламы (**Яндекс Директ, Google AdWords**) или таргетированной рекламы (**Vkontakte Target, myTarget от Mail.Ru** и т.д.) специалисты стали специализировать более узко - появились так называемые директологи(специалисты по контекстной рекламе), таргетологи(специалисты по таргетированной рекламе) и т.д.

В настоящее время для любого специалиста в области интернет-рекламы постоянно стоит задача оптимизации своих рекламных кампании, ввиду постоянно роста конкуренции, а следовательно и стоимости рекламы. Такая задача обычно решается с помощью анализа статистической информации о посетителях интернет-площадки, на которой происходит как торговля продуктом (интрнет-магазины), так и создание спроса на продукт и плавное подведение посетителя к его покупке (так называемые посадочные страницы - **[landing pages](https://ru.wikipedia.org/wiki/%D0%A6%D0%B5%D0%BB%D0%B5%D0%B2%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0)**)

Статистика обычно собирается с помощью двух наиболее популярных систем аналитики - Яндекс.Метрика или Google.Analytics. Затем специалист по интернет-рекламе проводит анализ данной статистики с целью поиска закономерностей, позволяющих скорректировать параметры рекламных кампаний таким образом, чтобы повысить их эффективность.

Вот пример из одного из моих реальных проектов:

<center>
<img src="img/1.jpg">

В данном случае в системе аналитики слева в столбце закодирована информация о таких настраиваемых параметрах рекламных кампаний, как формат банера, устройства для отображения, формат оплаты рекламы, дополнительные таргетинги интересов, возраст и пол посетителя. Такие комбинации параметров (а по сути на языке машинного обучения - это подпространство признаков) генерируются исключительно исходя из опыта того или иного специалиста. По приведенному выше скриншоту видно, что данный "срез" аналитики позволяет выявить, различия в показателе качества рекламы ([конверсии](https://goo.gl/xwstfU)) в 4! раза для некоторых комбинаций параметров. Таким образом появляется возможность "отключить" неэффективные рекламыне кампании (комбинации параметров) и повысить общую эффективность рекламного проекта. 

Целью данного исследования является максимальное прикладное использование машинного обучения с целью **получения от обученной модели топ-признаков с их весами, которые могут быть легко интерпретированны, как настраиваемые параметры рекламных кампаний, вносящие наибольший вклад в достижение целевого действия на посещемом сайте**.